In [ ]:
# Instala estas librerías si no las tienes
!pip install  transformers dataset evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np

import evaluate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import Dataset, DatasetDict


## cargar dataset

Descomprimir archivo zip

In [ ]:
!unzip "/content/reseñas_hoteles.zip" -d "/content/reseñas_hoteles"

Archive:  /content/reseñas_hoteles.zip
  inflating: /content/reseñas_hoteles/reseñas_hoteles.csv  


In [ ]:
# Cargar el dataset
csv_path = "/content/reseñas_hoteles/reseñas_hoteles.csv"
df = pd.read_csv(csv_path)

# Asegurarse de que las clases estén entre 0 y 4 (para 5 clases)
df["label"] = df["label"] - 1  # de 0 a 4
df = df[["reseña", "label"]]
df.head()


,reseña,label
0,No fue lo que esperaba. Algunos servicios no e...,1
1,"La habitación era aceptable, pero el ruido fue...",1
2,Excelente hotel. Superó todas mis expectativas.,4
3,Buena experiencia. La atención fue excelente a...,3
4,El servicio podría mejorar bastante. No lo rec...,1


## Acondicionar columnas

Separar los dataset en train, test y validation.

df_train_es (70% del dataset original)
df_test_es (20% del dataset original)
df_val_es (10% del dataset original)

In [ ]:
from sklearn.model_selection import train_test_split

# Paso 1: Dividir en entrenamiento (70%) y el resto (30%)
df_train_es, df_temp_es = train_test_split(df, test_size=0.3, random_state=42)  # Usa un random_state para reproducibilidad

# Paso 2: Dividir el resto (30%) en prueba (20%) y validación (10%)
df_test_es, df_val_es = train_test_split(df_temp_es, test_size=2/3, random_state=42)  # 20/30 = 2/3

## Adaptar formato del dataset 🔧

In [ ]:
# Convertir los DataFrames en objetos Dataset de la librería datasets
train_dataset = Dataset.from_pandas(df_train_es)
test_dataset = Dataset.from_pandas(df_test_es)
val_dataset = Dataset.from_pandas(df_val_es)

# Crear un DatasetDict con los conjuntos de datos
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Ver la estructura
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['reseña', 'label', '__index_level_0__'],
        num_rows: 102900
    })
    validation: Dataset({
        features: ['reseña', 'label', '__index_level_0__'],
        num_rows: 29400
    })
    test: Dataset({
        features: ['reseña', 'label', '__index_level_0__'],
        num_rows: 14700
    })
})


## Tokenización 📊

In [ ]:
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained(
   # "PlanTL-GOB-ES/roberta-base-bne",
   # local_files_only=False  # Fuerza descarga local, no inferencia
#)

In [ ]:

model_checkpoint = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, local_files_only=False)  # Fuerza descarga local, no inferencia)








/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
# Tokenizar función
def tokenize_function(examples):
    return tokenizer(examples["reseña"], truncation=True)

In [ ]:
columns = dataset["train"].column_names
columns.remove("label")
encoded_dataset = dataset.map(tokenize_function, batched=True, remove_columns=columns)
print(encoded_dataset)

Map:   0%|          | 0/102900 [00:00<?, ? examples/s]

Map:   0%|          | 0/29400 [00:00<?, ? examples/s]

Map:   0%|          | 0/14700 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 102900
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 29400
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 14700
    })
})


## Finetuning de la convnet

In [ ]:
# Modelo BERT para clasificación de 5 clases
model = BertForSequenceClassification.from_pretrained("PlanTL-GOB-ES/roberta-base-bne", num_labels=5)


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer

Vamos a evaluar el accuracy como métrica de rendimiento

In [ ]:
metric = evaluate.load("accuracy")
print(metric)

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

## Hugging Face Hub 🤗

El modelo entrenado lo vamos a subir a Hugging Face Hub así lo podemos compartir con el mundo 😎

Importante: La nueva credencial que vamos a crear debe tener permisos de escritura (write).

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `General` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `General`


In [ ]:
!git config --global credential.helper store

## Entrenamiento 💪

In [ ]:
model_name = model_checkpoint.split("/")[-1]

In [ ]:
print(model_name)

roberta-base-bne


In [ ]:
batch_size = 8
num_train_epochs=2
num_train_samples = len(encoded_dataset["train"])
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

In [ ]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    hub_model_id=f"{model_name}-resenas-project-nlp-con-transformers"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    processing_class=tokenizer,
)

Para este paso es necesario crearse una cuenta gratuita en Weights & Biases, porque el entrenamiento y las métricas se harán ahí.

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sandrarairan (sandrarairan-sr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.000000,1.000000
2,0.000000,0.000000,1.000000


TrainOutput(global_step=25726, training_loss=0.001415070242137549, metrics={'train_runtime': 2220.5048, 'train_samples_per_second': 92.682, 'train_steps_per_second': 11.586, 'total_flos': 1761847304380896.0, 'train_loss': 0.001415070242137549, 'epoch': 2.0})

## Guardar el modelo 💾

Para eso vamos a hacer un push a Hugging Face Hub

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1748984629.3e5f35561eb2.767.0:   0%|          | 0.00/7.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sanrapi/roberta-base-bne-resenas-project-nlp-con-transformers/commit/5bd1cf41e4098d0badecc4999cfdb749a4706910', commit_message='End of training', commit_description='', oid='5bd1cf41e4098d0badecc4999cfdb749a4706910', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sanrapi/roberta-base-bne-resenas-project-nlp-con-transformers', endpoint='https://huggingface.co', repo_type='model', repo_id='sanrapi/roberta-base-bne-resenas-project-nlp-con-transformers'), pr_revision=None, pr_num=None)

##  Hacer Predicciones en Producción 🤙

In [ ]:
from transformers import pipeline

Cargar el modelo una vez (al inicio de la aplicación)

In [ ]:
model_checkpoint = "sanrapi/roberta-base-bne-resenas-project-nlp-con-transformers"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

config.json:   0%|          | 0.00/959 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Device set to use cuda:0


## Ejemplos de uso

In [ ]:
pipe("desagradable paseo.La piscina muy sucia!!!")

[{'label': 'LABEL_3', 'score': 0.9999769926071167}]

In [ ]:
pipe("la peor compra de mi vida!!! no recomiendo!")

[{'label': 'LABEL_1', 'score': 0.9748082160949707}]